In [1]:
!pip install azure-storage-blob azure-identity    # 애져 패키지 설치(블롭 스토리지용 패키지)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.6 MB/s eta 0:00:00


In [3]:
# 일반적으로 가장 많이 사용하는 blob 스토리지
# 스토리지 계정을 만들어 사용.
# 컨테이너를 만들어 대용량 데이터를 저장할 때 사용
# 단일 블록 blob에는 최대 50000개 블록 저장 가능

import os, uuid                                           
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


In [4]:
# Blob Service에 접속할 때 접속 문자열 필요
connect_str = 'DefaultEndpointsProtocol=https;AccountName=labuser112;AccountKey=CrcGeR+RVqmDWjp01BhiAPHHqQI1HGOIqfGNJQ7icmZyumAqi7w68NFSykhLTODTOlsYKF2OcfPN+AStfGfg0w==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [5]:
# 컨테이너 생성
container_name = 'test2container'

container_client = blob_service_client.create_container(container_name)

In [6]:
# Create a local directory to hold blob data
local_path = './data'
os.mkdir(local_path)

# Create a file in the local data directory to upload and download
local_file_name = 'minsol' + '.txt'
upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
file = open(upload_file_path, mode='w')
file.write('Hi, I am min sol')            
file.close()                                  # 파일 편집 후 반드시 닫아줘야 나중에 깨지지 않는다.

In [7]:
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

with open(file=upload_file_path, mode='rb') as data:          # 텍스트(메모장)사용자 친화, 바이너리(exe/bin) 효율적 동작 
  blob_client.upload_blob(data)                                         # w = 쓰기, r = 읽기,  뒤에 b 붙으면 바이너리형태로 실행하겠다는 뜻
# 텍스트 형태로 올리면 깨질 가능성이 있기 때문에 바이너리형태로 올리는 것
# 텍스트는 어느 단위로 편집하던 상관없다.

In [10]:
 # blob 스토리지 안의 파일 목록 확인
print('\nListing blobs')

blob_list = container_client.list_blobs()      #blob 목록 불러오기
for blob in blob_list:
  print('\t' + blob.name)


Listing blobs
	minsol.txt


In [14]:
# blob 스토리지의 파일을 로컬로 다운받는 법
# 원본 파일과 겹치지 않게 다운로드라는 키워드를 붙이기

download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name, '.txt', '_Download.txt'))      # 기존 파일명 변경(치환 개념)
print(download_file_path)                                  
container_client = blob_service_client.get_container_client(container=container_name)

with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())             # 클라우드의 파일 다운로드 받고 읽어들여서 다운로드파일에다가 작성까지


./data/minsol_Download.txt


In [15]:
# Clean Up
print('\nPress the Enter key to begin clean up')
input()                     # 사용자의 입력을 기다리는 창 띄우기

print('Deleting blob container.....')
container_client.delete_container()

print('Deleting the local source and downloded files....')
os.remove(upload_file_path)                 # 업로드 대상 파일 지우기
os.remove(download_file_path)             # 다운로드 파일 지우기
os.rmdir(local_path)                             # 만들었던 디렉토리 지우기

print('Done.')



Press the Enter key to begin clean up

Deleting blob container.....
Deleting the local source and downloded files....
Done.


In [ ]:
# 가상 머신의 디스크는 언제라도 초기화될수있다. 클라우드 관리자에 의해 삭제될수있다.
# 스토리지 계정을 따로 만들어야 함.
# blob이나 file 스토리지를 이용해 본인 클라우드에 있는 파일을 참조하는 형식으로 사용해야한다.

# file 스토리지 = 포트넘버 445, SMB 3.0(네트워크 파일 공유 프로토콜),
# 드라이브 공유(드라이브를 사용하고 있는 사용자끼리 파일 공유 가능)
# blob 스토리지 = 별도의 저장공간 느낌
# RDP(Remote Desktop Protocol):3389  = 윈도우 전용 원격 데스크탑 프로토콜

